# Table Management Lab

## Lab Objectives
By the end of this lab, you will be able to:
- Create tables with proper data types and constraints
- Alter table structures using various ALTER TABLE operations
- Manage constraints (PRIMARY KEY, FOREIGN KEY, UNIQUE, CHECK)
- Add and manage indexes for performance
- Rename and drop tables safely
- Perform table maintenance operations
- Understand table storage engines and options

## Prerequisites
- Basic SQL knowledge (CREATE, SELECT, INSERT)
- Understanding of data types and constraints
- MySQL Server installed and running

## Lab Duration
Approximately 90 minutes

## Materials Needed
- MySQL Server
- Python environment with mysql-connector-python
- This Jupyter notebook

## Table Management Concepts

### Table Creation
- **CREATE TABLE**: Define table structure with columns, data types, and constraints
- **Data Types**: INT, VARCHAR, TEXT, DATE, DECIMAL, BOOLEAN, etc.
- **Constraints**: PRIMARY KEY, FOREIGN KEY, UNIQUE, NOT NULL, CHECK, DEFAULT
- **Table Options**: ENGINE, CHARSET, AUTO_INCREMENT, etc.

### Table Alteration
- **ADD COLUMN**: Add new columns to existing tables
- **MODIFY COLUMN**: Change column data types and constraints
- **DROP COLUMN**: Remove columns from tables
- **CHANGE COLUMN**: Rename columns and modify their definitions

### Constraint Management
- **ADD CONSTRAINT**: Add various types of constraints
- **DROP CONSTRAINT**: Remove constraints
- **Constraint Types**: PRIMARY KEY, FOREIGN KEY, UNIQUE, CHECK

### Index Management
- **CREATE INDEX**: Add indexes for query performance
- **DROP INDEX**: Remove unused indexes
- **Index Types**: BTREE, FULLTEXT, SPATIAL

### Table Operations
- **RENAME TABLE**: Change table names
- **DROP TABLE**: Permanently remove tables
- **TRUNCATE TABLE**: Remove all data but keep structure
- **Table Maintenance**: ANALYZE, OPTIMIZE, REPAIR

## Step-by-Step Guide

First, install the required Python package:

In [ ]:
!pip install mysql-connector-python

## Step 1: Connect to MySQL and Set Up Database

Connect to MySQL and create the practice database.

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='your_password'
)
cursor = conn.cursor()

# Create database
cursor.execute('CREATE DATABASE IF NOT EXISTS table_management_lab')
cursor.execute('USE table_management_lab')
print('Database ready for table management practice')

## Step 2: Create Basic Tables

Start by creating tables with various data types and constraints.

In [ ]:
# Create a basic users table
cursor.execute('''
CREATE TABLE users (
    user_id INT,
    username VARCHAR(50),
    email VARCHAR(100),
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    date_of_birth DATE,
    is_active BOOLEAN DEFAULT TRUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

print('Users table created successfully')

# Check table structure
cursor.execute('DESCRIBE users')
columns = cursor.fetchall()
print('\nUsers table structure:')
print('-' * 50)
for col in columns:
    nullable = 'YES' if col[2] == 'YES' else 'NO'
    default = col[4] if col[4] else 'NULL'
    print(f'{col[0]:<15} | {col[1]:<10} | {nullable:<3} | {default}')

In [ ]:
# Create products table with constraints
cursor.execute('''
CREATE TABLE products (
    product_id INT NOT NULL,
    product_name VARCHAR(100) NOT NULL,
    description TEXT,
    price DECIMAL(10, 2) NOT NULL CHECK (price > 0),
    stock_quantity INT DEFAULT 0 CHECK (stock_quantity >= 0),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE (product_name),
    PRIMARY KEY (product_id)
)
''')

print('Products table created with constraints')

# Insert sample data to test constraints
products_data = [
    (1, 'Laptop', 'High-performance laptop', 1299.99, 50),
    (2, 'Mouse', 'Wireless optical mouse', 29.99, 200),
    (3, 'Keyboard', 'Mechanical keyboard', 89.99, 100)
]

cursor.executemany('''
INSERT INTO products (product_id, product_name, description, price, stock_quantity)
VALUES (%s, %s, %s, %s, %s)
''', products_data)

conn.commit()
print(f'Inserted {cursor.rowcount} products')

## Step 3: Alter Table Structures

Practice adding, modifying, and dropping columns.

In [ ]:
# Add single column
cursor.execute('''
ALTER TABLE users
ADD COLUMN phone VARCHAR(20)
''')

print('Added phone column to users table')

# Add multiple columns
cursor.execute('''
ALTER TABLE users
ADD COLUMN middle_name VARCHAR(50),
ADD COLUMN profile_picture VARCHAR(255),
ADD COLUMN last_login TIMESTAMP
''')

print('Added middle_name, profile_picture, and last_login columns')

# Verify changes
cursor.execute('DESCRIBE users')
columns = cursor.fetchall()
print('\nUpdated users table structure:')
print('-' * 60)
for col in columns:
    nullable = 'YES' if col[2] == 'YES' else 'NO'
    default = col[4] if col[4] else 'NULL'
    print(f'{col[0]:<18} | {col[1]:<12} | {nullable:<3} | {default}')

In [ ]:
# Modify column data type
cursor.execute('''
ALTER TABLE users
MODIFY COLUMN phone VARCHAR(25)
''')

print('Modified phone column to VARCHAR(25)')

# Modify column with constraints
cursor.execute('''
ALTER TABLE users
MODIFY COLUMN email VARCHAR(150) NOT NULL
''')

print('Modified email column to NOT NULL')

# Change column name and type
cursor.execute('''
ALTER TABLE users
CHANGE COLUMN date_of_birth birth_date DATE NOT NULL
''')

print('Renamed date_of_birth to birth_date and made it NOT NULL')

In [ ]:
# Drop columns
cursor.execute('''
ALTER TABLE users
DROP COLUMN middle_name
''')

print('Dropped middle_name column')

# Drop multiple columns
cursor.execute('''
ALTER TABLE users
DROP COLUMN profile_picture,
DROP COLUMN last_login
''')

print('Dropped profile_picture and last_login columns')

# Final structure check
cursor.execute('DESCRIBE users')
columns = cursor.fetchall()
print('\nFinal users table structure:')
print('-' * 55)
for col in columns:
    nullable = 'YES' if col[2] == 'YES' else 'NO'
    default = col[4] if col[4] else 'NULL'
    print(f'{col[0]:<15} | {col[1]:<12} | {nullable:<3} | {default}')

## Step 4: Manage Constraints

Add and remove various types of constraints.

In [ ]:
# Add primary key to users table
cursor.execute('''
ALTER TABLE users
ADD PRIMARY KEY (user_id)
''')

print('Added PRIMARY KEY constraint to users table')

# Add unique constraint
cursor.execute('''
ALTER TABLE users
ADD CONSTRAINT uk_users_email UNIQUE (email)
''')

print('Added UNIQUE constraint on email column')

# Add check constraint
cursor.execute('''
ALTER TABLE products
ADD CONSTRAINT chk_positive_stock CHECK (stock_quantity >= 0)
''')

print('Added CHECK constraint for positive stock quantity')

In [ ]:
# Create categories table for foreign key relationship
cursor.execute('''
CREATE TABLE categories (
    category_id INT PRIMARY KEY AUTO_INCREMENT,
    category_name VARCHAR(100) NOT NULL UNIQUE,
    description TEXT
)
''')

# Add foreign key constraint
cursor.execute('''
ALTER TABLE products
ADD COLUMN category_id INT,
ADD CONSTRAINT fk_products_category
FOREIGN KEY (category_id) REFERENCES categories(category_id)
''')

print('Added category_id column and FOREIGN KEY constraint to products table')

# Insert category data
categories_data = [
    ('Electronics', 'Electronic devices and accessories'),
    ('Computers', 'Computer hardware and software'),
    ('Accessories', 'Computer accessories')
]

cursor.executemany('''
INSERT INTO categories (category_name, description)
VALUES (%s, %s)
''', categories_data)

conn.commit()
print(f'Inserted {cursor.rowcount} categories')

In [ ]:
# Test constraint violations
try:
    # Try to insert duplicate email (unique constraint)
    cursor.execute('''
    INSERT INTO users (user_id, username, email, first_name)
    VALUES (1, 'test', 'duplicate@email.com', 'Test')
    ''')
    cursor.execute('''
    INSERT INTO users (user_id, username, email, first_name)
    VALUES (2, 'test2', 'duplicate@email.com', 'Test2')
    ''')
    conn.commit()
except mysql.connector.Error as err:
    print(f'Expected UNIQUE constraint violation: {err}')
    conn.rollback()

try:
    # Try to insert invalid category_id (foreign key constraint)
    cursor.execute('''
    UPDATE products
    SET category_id = 999
    WHERE product_id = 1
    ''')
    conn.commit()
except mysql.connector.Error as err:
    print(f'Expected FOREIGN KEY constraint violation: {err}')
    conn.rollback()

print('Constraint testing completed')

## Step 5: Manage Indexes

Add and manage indexes for performance.

In [ ]:
# Add various indexes
cursor.execute('''
ALTER TABLE products
ADD INDEX idx_product_name (product_name),
ADD INDEX idx_price (price),
ADD INDEX idx_category_price (category_id, price)
''')

print('Added indexes on products table')

# Show indexes
cursor.execute('SHOW INDEXES FROM products')
indexes = cursor.fetchall()
print('\nProducts table indexes:')
print('-' * 70)
for idx in indexes:
    print(f'{idx[2]:<20} | {idx[4]:<15} | {idx[7] if idx[7] else "N/A":<10} | {"Unique" if idx[1] == "0" else "Non-unique"}')

## Step 6: Rename and Drop Tables

Practice safe table renaming and dropping operations.

In [ ]:
# Create a temporary table to demonstrate renaming
cursor.execute('''
CREATE TABLE temp_data (
    id INT PRIMARY KEY,
    data VARCHAR(100)
)
''')

# Insert some data
cursor.execute("INSERT INTO temp_data VALUES (1, 'Sample data')")
conn.commit()

print('Created temporary table with data')

# Rename table
cursor.execute('RENAME TABLE temp_data TO archived_data')
print('Renamed temp_data to archived_data')

# Verify rename
cursor.execute('SELECT * FROM archived_data')
result = cursor.fetchone()
print(f'Data in renamed table: {result}')

# Drop table safely
cursor.execute('DROP TABLE IF EXISTS archived_data')
print('Dropped archived_data table')

## Step 7: Table Maintenance Operations

Perform maintenance operations on tables.

In [ ]:
# Analyze table (update statistics)
cursor.execute('ANALYZE TABLE products')
print('Analyzed products table')

# Show table status
cursor.execute("SHOW TABLE STATUS LIKE 'products'")
status = cursor.fetchone()
print(f'\nTable status for products:')
print(f'Engine: {status[1]}')
print(f'Rows: {status[4]}')
print(f'Avg row length: {status[5]}')
print(f'Data length: {status[6]} bytes')
print(f'Index length: {status[7]} bytes')

# Check table integrity
cursor.execute('CHECK TABLE products')
check_result = cursor.fetchone()
print(f'\nTable check result: {check_result[3]}')

## Step 8: Advanced Table Operations

Practice advanced table management techniques.

In [ ]:
# Create table from SELECT (copy structure and data)
cursor.execute('''
CREATE TABLE products_backup AS
SELECT * FROM products
''')

print('Created products_backup table from SELECT')

# Create summary table
cursor.execute('''
CREATE TABLE products_summary (
    category_name VARCHAR(100),
    product_count INT,
    total_value DECIMAL(12, 2),
    avg_price DECIMAL(8, 2)
)
''')

print('Created products_summary table')

# Populate summary with aggregated data
cursor.execute('''
INSERT INTO products_summary
SELECT 
    COALESCE(c.category_name, 'Uncategorized') as category_name,
    COUNT(p.product_id) as product_count,
    SUM(p.price * p.stock_quantity) as total_value,
    AVG(p.price) as avg_price
FROM products p
LEFT JOIN categories c ON p.category_id = c.category_id
GROUP BY c.category_name
''')

conn.commit()
print(f'Inserted {cursor.rowcount} summary records')

In [ ]:
# Query the summary table
cursor.execute('SELECT * FROM products_summary')
results = cursor.fetchall()
print('Products summary:')
print('-' * 70)
for row in results:
    print(f'{row[0]:<15} | {row[1]:<5} products | ${row[2]:>10,.2f} | ${row[3]:>8,.2f}')

## Step 9: Schema Migration Example

Demonstrate safe schema changes with data migration.

In [ ]:
# Add new column with default
cursor.execute('''
ALTER TABLE users
ADD COLUMN email_verified BOOLEAN DEFAULT FALSE
''')

print('Added email_verified column with default FALSE')

# Simulate data migration (mark .com emails as verified)
cursor.execute('''
UPDATE users
SET email_verified = TRUE
WHERE email LIKE '%.com'
''')

print(f'Updated {cursor.rowcount} users with .com emails as verified')

# Now make the column NOT NULL
cursor.execute('''
ALTER TABLE users
MODIFY COLUMN email_verified BOOLEAN NOT NULL DEFAULT FALSE
''')

conn.commit()
print('Made email_verified column NOT NULL')

# Verify the migration
cursor.execute('SELECT COUNT(*) as verified_count FROM users WHERE email_verified = TRUE')
verified_count = cursor.fetchone()[0]
cursor.execute('SELECT COUNT(*) as total_count FROM users')
total_count = cursor.fetchone()[0]
print(f'\nMigration results: {verified_count}/{total_count} users have verified emails')

## Step 10: Clean Up

Close the database connection.

In [ ]:
cursor.close()
conn.close()
print('Database connection closed')

## Lab Summary

Excellent! You have successfully completed the Table Management Lab. In this lab, you learned how to:

1. **Create Tables**: With proper data types, constraints, and options
2. **Alter Structures**: Add, modify, and drop columns safely
3. **Manage Constraints**: Add and remove PRIMARY KEY, FOREIGN KEY, UNIQUE, and CHECK constraints
4. **Handle Indexes**: Create and manage indexes for performance
5. **Rename Tables**: Change table names using ALTER TABLE and RENAME TABLE
6. **Drop Tables**: Remove tables safely with proper checks
7. **Perform Maintenance**: Analyze, optimize, and repair tables
8. **Advanced Operations**: Create tables from SELECT, schema migrations

## Key Concepts Learned
- **CREATE TABLE**: Complete table definition with constraints and options
- **ALTER TABLE**: Modify existing table structures
- **ADD/DROP CONSTRAINT**: Manage data integrity rules
- **CREATE/DROP INDEX**: Optimize query performance
- **RENAME/DROP TABLE**: Table lifecycle management
- **ANALYZE/OPTIMIZE**: Table maintenance operations
- **Schema Migration**: Safe database structure changes

## Best Practices
- Always backup before major schema changes
- Test ALTER operations on development environment first
- Use descriptive constraint and index names
- Plan schema changes to minimize downtime
- Document all table structure changes
- Consider performance impact of schema changes
- Use transactions for multiple related changes

## Common DDL Patterns
```sql
-- Safe column addition
ALTER TABLE table_name ADD COLUMN new_col datatype DEFAULT value;

-- Constraint addition
ALTER TABLE table_name ADD CONSTRAINT ck_name CHECK (condition);

-- Index creation
ALTER TABLE table_name ADD INDEX idx_name (column);

-- Safe table drop
DROP TABLE IF EXISTS table_name;

-- Table maintenance
ANALYZE TABLE table_name;
OPTIMIZE TABLE table_name;
```

## Table Creation Checklist
- [ ] Appropriate data types selected
- [ ] Primary key defined
- [ ] Foreign keys established
- [ ] Check constraints added
- [ ] Default values specified
- [ ] Indexes planned for performance
- [ ] Table options configured

## Schema Change Safety
1. **Backup**: Always backup before changes
2. **Test**: Test on development/staging first
3. **Plan**: Schedule during maintenance windows
4. **Document**: Record all changes made
5. **Verify**: Test application functionality after changes
6. **Monitor**: Watch for performance impacts

## Advanced Topics to Explore
- Database normalization (1NF, 2NF, 3NF)
- Partitioning strategies for large tables
- Online DDL operations (MySQL 8.0+)
- Schema versioning and migration tools
- Performance monitoring and tuning
- High availability considerations

## Performance Considerations
- Add indexes on frequently queried columns
- Use appropriate data types to save space
- Consider table partitioning for large datasets
- Monitor slow query logs after schema changes
- Use EXPLAIN to analyze query performance
- Consider denormalization for read-heavy workloads

## Next Steps
- Learn about database design and ER modeling
- Study query optimization and indexing strategies
- Explore stored procedures and triggers
- Learn about database backup and recovery
- Study database security and access control

## Challenge Exercises

### Challenge 1: Complete Schema Design
Design and implement a complete e-commerce database schema including:
- Users, products, categories, orders, order_items
- Proper relationships and constraints
- Indexes for performance
- Data validation rules

### Challenge 2: Schema Migration
Create a migration script that safely:
- Adds audit columns (created_at, updated_at)
- Implements soft deletes (is_deleted flag)
- Adds new relationships
- Migrates existing data

### Challenge 3: Performance Optimization
Analyze and optimize a poorly designed schema:
- Identify missing indexes
- Suggest data type improvements
- Propose partitioning strategies
- Implement performance monitoring

Remember: Table management is a critical skill for database administrators. Proper table design and maintenance ensure data integrity, performance, and scalability of your database systems!